In [ ]:
# Imports
import os
import sys
import glob
import re

# Third party imports
import numpy as np
import xarray as xr
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [ ]:
# Import dictionaries
sys.path.append('/home/users/benhutch/skill-maps')
import dictionaries as dict

# Import functions
import functions as func

# Import nao matching functions
sys.path.append('/home/users/benhutch/skill-maps/rose-suite-matching')
from nao_matching_seasons import match_variable_models, find_obs_path

### NAO skill exploration ###

* Create a subplot with 6 rows and 2 columns, containing the NAO 8-year running means (years 2-9 forecast) for psl.
* Shift the x-axis to be aligned with initialisation year
* Grab as many years as possible (final 8 year running mean for obs 2016-2023 inclusive)
* Present in terms of long region and short region
* Also consider temperature of subpolar gyre for different models

In [ ]:
# Set up the arguments for psl
psl_models = dict.models
obs_path_psl = dict.obs
base_dir = dict.base_dir
plots_dir = "/gws/nopw/j04/canari/users/benhutch/plots/NAO_skill"
save_dir = dict.save_dir